In [2]:
import struct
import datetime as dt
import glob
import os
from collections import namedtuple
import matplotlib.image as mpimg
import cv2

import numpy as np
# import pykitti.utils as utils

In [3]:
# fileName = '/Users/GF/Downloads/data_road_velodyne/testing/velodyne/um_000000.bin'
# with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
# struct.unpack("iiiii", fileContent[:20])

In [36]:
def get_velo_scans(velo_file):
    """read velodyne [x,y,z,reflectance] scan data from binary files."""
    """parse velodyne binary files into arrays."""
    scan = np.fromfile(velo_file, dtype='float32')
    return scan.reshape((-1, 4))
        
# def velo(data_path):
#         """Generator to read velodyne [x,y,z,reflectance] scan data from binary files."""
#         # Find all the Velodyne files
#         velo_path = os.path.join(
#             data_path, 'velodyne_points', 'data', '*.bin')
#         velo_files = sorted(glob.glob(velo_path))

#         # Subselect the chosen range of frames, if any
# #         if self.frames is not None:
# #             velo_files = [velo_files[i] for i in self.frames]

#         # Return a vgenerator yielding Velodyne scans.
#         # Each scan is a Nx4 array of [x,y,z,reflectance]
#         return get_velo_scans(velo_files)
def sobel(img, thresh_min=0, thresh_max=255): # use the threshold values you found to replace 0 and 255
    #1. convert the image to gray scale
    #2. Gaussian blur the image
    #3. Use Sobel to find derievatives for both X and Y Axis
    #4. Use addweight to combine the results
    #5. Convert each pixel to unint8, then apply threshold to get binary image
    ## TODO
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    #name your result to be sobelX and sobelY
    sobelX = cv2.Sobel(img_blur,cv2.CV_8U,1,0,3)
    sobelY = cv2.Sobel(img_blur,cv2.CV_8U,0,1,3)
    
    ####
    
    combined = cv2.addWeighted(sobelX, 0.9, sobelY, 0.1, 0)
    #scaled_lap = np.uint8(255*combined/np.max(combined)) #for some reason, scaled_lap was only containing 0s and 2s
    binary_output = np.zeros_like(combined)
    binary_output[(combined >= thresh_min) & (combined <= thresh_max)] = 1
    #for i in range(0,len(scaled_lap)):
    #    for j in range(0,len(scaled_lap[i])):
    #        print(scaled_lap[i][j])

    return binary_output
print ('...')

...


In [64]:
#filename = '/Users/GF/Downloads/data_road_velodyne-2/training/velodyne/um_000003.bin'
filename = '/Users/Mike/Desktop/data_road_velodyne/training/velodyne/um_000022.bin'
camera_filename = '/Users/Mike/Desktop/data_road_gray/training/image_0/um_000022.png'
cam_img = np.fromfile(camera_filename, dtype='float32') #cv2.imread(camera_filename)
arr = get_velo_scans(filename)
new_arr = np.full((375,1242,4),(0,0,0,255))
for point in arr:
    if(point[3] > 0 and point[0] > 0 and point[0] <= 37.4 and abs(point[1]) < 9.95 and point[2] < 0):
        x_coord = int(round(point[1]*20)+200)
        y_coord = int(round(point[0]*10))
        new_arr[374-y_coord][820-x_coord] = (0,int(point[3]*255),0,255)
cv2.imwrite("out22.png", new_arr)
img = np.float32(new_arr)

#thresh_min1 = 0
#thresh_max1 = 255
#bw_img = sobel(img, thresh_min1, thresh_max1)
#cv2.imwrite("out3_bw.png", bw_img)

#warp perspctive
x3, y3 = 655, 299
x1, y1 = 657, 243
x4, y4 = 518, 300
x2, y2 = 516, 248

#400,400
#1242,375 .9375

dst = np.float32([(708,260), #109 #279
                  (343,263), #150 101
                  (788,332), #284
                  (165,325)])   
src = np.float32([(x1,y1),(x2,y2),(x3,y3),(x4,y4)])

M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst,src)
img = np.float32(new_arr)
warpedImg = cv2.warpPerspective(img, M, (1242,375), flags=cv2.INTER_LINEAR)
cam_img = np.float32(cam_img)
#newImg = cv2.addWeighted(cam_img, 1, warpedImg, .5, 0)
#for i in range(1242):
#    for j in range(375):
#        if(warpedImg[j][i][1] > 0):
#            cam_img[j][i] = warpedImg[j][i];
cv2.imwrite("out22_w.png", warpedImg)
#cv2.imwrite("out3_comb.png", camImg)
        
# scan = np.fromfile(filename, dtype='uint8')
# cv2.imwrite('scan.png', scan[:640000].reshape(64,10000))

True